## ПИ21-1В
## Сабиров А.А.

In [ ]:
%pip install pandarallel

%pip install pymorphy3

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.1 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16672 sha256=343bbaedf451eedbdff5f7ebecbeb82c6e83aba2ca6d82b7135d7266257482d4
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 57.8 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import re

import pandas as pd
import numpy as np

from collections import Counter
from itertools import chain


from tqdm import tqdm
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=8)


from nltk.metrics.distance import edit_distance
from nltk.stem import SnowballStemmer

import pymorphy3


pd.set_option('display.max_colwidth', None)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
#ИНИЦИАЛИЗАЦИЯ СПИСКА РУССКИХ СТОП-СЛОВ

stop_words = nltk.corpus.stopwords.words('russian')

stop_words[:5], stop_words[-5:]

(['и', 'в', 'во', 'не', 'что'], ['более', 'всегда', 'конечно', 'всю', 'между'])

In [ ]:
pushkin = pd.read_csv("https://raw.githubusercontent.com/ibbvs/jasa/main/TOBD-labs/data%20sources/PUSHKINS.TXT",
                 encoding="windows-1251",
                 delimiter="\t",
                 skiprows=1300)    #skiprows=17


#df.to_csv("answer.txt", sep='\t', index=False)

pushkin.head()

,СТИХОТВОРЕНИЯ 1813
0,К НАТАЛЬЕ.
1,Pourquoi craindrais-j'e de ie dire?
2,C'est Margot qui fixe mon goъt.
3,"Так и мне узнать случилось,"
4,Что за птица Купидон;


#----------------------------------------------------------------------------------------------------------------------------


In [ ]:
pushkin.iloc[-2]

СТИХОТВОРЕНИЯ 1813      Глаза ее так и блистали, голос так и звенел. Я обняла ее, мы смешали слезы благородного восторга, и жаркие моления за отечество. Ты не знаешь? сказала мне Полина с видом вдохновенн<ым>. - Твой брат.... он счастлив, он не в плену - радуйся: он убит за спасение России.
Name: 30480, dtype: object

СТИХОТВОРЕНИЯ 1813      Глаза ее так и блистали, голос так и звенел. Я обняла ее, мы смешали слезы благородного восторга, и жаркие моления за отечество. Ты не знаешь? сказала мне Полина с видом `вдохновенн<ым>`. - Твой брат.... он счастлив, он не в плену - радуйся: он убит за спасение России.
Name: 30480, dtype: object

In [ ]:
pushkin = pushkin.parallel_applymap(lambda x: re.sub(r"<|>", "", x.lower().strip()))
pushkin.iloc[-2]

СТИХОТВОРЕНИЯ 1813    глаза ее так и блистали, голос так и звенел. я обняла ее, мы смешали слезы благородного восторга, и жаркие моления за отечество. ты не знаешь? сказала мне полина с видом вдохновенным. - твой брат.... он счастлив, он не в плену - радуйся: он убит за спасение россии.
Name: 30480, dtype: object

In [ ]:
#ОЧИСТКА СТРОК ОТ ЗНАКОВ ПУНКТУАЦИИ, ЦИФР, СПЕЦ.СИМВОЛОВ; УДАЛЕНИЕ ИНОСТРАННЫХ СЛОВ ПО ФЛАГУ
def remove_punkt(df, lang=False):
    if lang:
        return df.parallel_applymap(lambda x: re.sub(r"\W+|[^а-яА-ЯёЁ]", " ", x.lower().strip()))


    return df.parallel_applymap(lambda x: re.sub(r"[\W+|\d]+", " ", x.lower().strip()).strip())





pushkin = remove_punkt(pushkin, lang=True)
pushkin.iloc[-2]

СТИХОТВОРЕНИЯ 1813    глаза ее так и блистали голос так и звенел я обняла ее мы смешали слезы благородного восторга и жаркие моления за отечество ты не знаешь сказала мне полина с видом вдохновенным твой брат он счастлив он не в плену радуйся он убит за спасение россии 
Name: 30480, dtype: object

#----------------------------------------------------------------------------------------------------------------------------


In [ ]:
# СОХРАНЯЕМ ГРЯЗНЫЙ ФРЕЙМ, С СТОП-СЛОВАМИ

pushkin_with_stopwords = pushkin.copy()

pushkin_with_stopwords.iloc[-2]

СТИХОТВОРЕНИЯ 1813    глаза ее так и блистали голос так и звенел я обняла ее мы смешали слезы благородного восторга и жаркие моления за отечество ты не знаешь сказала мне полина с видом вдохновенным твой брат он счастлив он не в плену радуйся он убит за спасение россии 
Name: 30480, dtype: object

### --------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
%%time
# ФУНКЦИЯ ДЛЯ УДАЛЕНИЯ СТОП-СЛОВ ИЗ СТРОКИ
def remove_stopwords(text, stop_words):

    # Создаем регулярное выражение, которое удаляет стоп-слова и слова из одной буквы
    pattern = r'\b(?:{})\b|\b\w\b'.format('|'.join(map(re.escape, stop_words)))

    # Заменяем найденные слова на пустую строку
    return re.sub(pattern, '', text)



pushkin = pushkin.parallel_applymap(lambda x: remove_stopwords(x, stop_words))


print(f"СТРОКА ДО УДАЛЕНИЯ СТОП-СЛОВ:\n{pushkin_with_stopwords.iloc[-2]}\n\n\nСТРОКА ПОСЛЕ УДАЛЕНИЯ СТОП-СЛОВ:\n{pushkin.iloc[-2]}\n\n")

СТРОКА ДО УДАЛЕНИЯ СТОП-СЛОВ:
СТИХОТВОРЕНИЯ 1813    глаза ее так и блистали голос так и звенел я обняла ее мы смешали слезы благородного восторга и жаркие моления за отечество ты не знаешь сказала мне полина с видом вдохновенным твой брат он счастлив он не в плену радуйся он убит за спасение россии 
Name: 30480, dtype: object


СТРОКА ПОСЛЕ УДАЛЕНИЯ СТОП-СЛОВ:
СТИХОТВОРЕНИЯ 1813    глаза    блистали голос   звенел  обняла   смешали слезы благородного восторга  жаркие моления  отечество   знаешь сказала  полина  видом вдохновенным твой брат  счастлив    плену радуйся  убит  спасение россии 
Name: 30480, dtype: object


CPU times: user 216 ms, sys: 66.6 ms, total: 282 ms
Wall time: 2.42 s


In [ ]:
%%time
# ФУНКЦИЯ ПРИВЕДЕНИЯ pd.df К ОДНОМЕРНОМУ СПИСКУ
def transform_to_list(df):

    df = df.parallel_applymap(lambda x: nltk.word_tokenize(x))


    return list(chain.from_iterable(df[df.columns[0]].to_list()))


# СПИСОК ТОКЕНОВ
pushkin_tokens = transform_to_list(pushkin)


# СОЗДАНИЕ СЛОВАРЯ ГДЕ КЛЮЧ=СЛОВО, А ЗНАЧЕНИЕ=КОЛИЧЕСТВО ЕГО ПОВТОРЕНИЙ В ТЕКСТЕ
pushkin_dict = Counter(pushkin_tokens)


print(f"\nПОЛУЧЕННЫЕ ТОКЕНЫ:\n{pushkin_tokens[:5]}\n\n5 САМЫХ РАСПРОСТРАННЕННЫХ СЛОВ:\n{pushkin_dict.most_common(5)}\n\n")


ПОЛУЧЕННЫЕ ТОКЕНЫ:
['наталье', 'узнать', 'случилось', 'птица', 'купидон']

5 САМЫХ РАСПРОСТРАННЕННЫХ СЛОВ:
[('всё', 664), ('сказал', 449), ('это', 439), ('тебе', 380), ('день', 297)]


CPU times: user 265 ms, sys: 111 ms, total: 376 ms
Wall time: 2.5 s


In [ ]:
'''
%%time
# ФУНКЦИЯ ПРИВЕДЕНИЯ pd.df К ОДНОМЕРНОМУ СПИСКУ
def transform_to_list(df):
    lst = df[df.columns[0]].to_list()

    # ТОКЕНИЗАЦИЯ
    words = [nltk.word_tokenize(row) for row in lst]


    return [word for sublist in words for word in sublist]


# СПИСОК ТОКЕНОВ
pushkin_tokens = transform_to_list(pushkin)


# СОЗДАНИЕ СЛОВАРЯ ГДЕ КЛЮЧ=СЛОВО, А ЗНАЧЕНИЕ=КОЛИЧЕСТВО ЕГО ПОВТОРЕНИЙ В ТЕКСТЕ
pushkin_dict = Counter(pushkin_tokens)


print(f"ПОЛУЧЕННЫЕ ТОКЕНЫ:\n{pushkin_tokens[:5]}\n\n5 САМЫХ РАСПРОСТРАННЕННЫХ СЛОВ:\n{pushkin_dict.most_common(5)}\n\n")
'''

#---------------------------------------------------------------------------------------------------------------

In [ ]:
# ГРЯЗНЫЙ ФРЕЙМ, С СТОП-СЛОВАМИ
pushkin_with_stopwords.head()

,СТИХОТВОРЕНИЯ 1813
0,к наталье
1,
2,ъ
3,так и мне узнать случилось
4,что за птица купидон


In [ ]:
%%time
# ПРИВЕДЕНИЕ ГРЯЗНОГО ФРЕЙМА К ОДНОМЕРНОМУ СПИСКУ, СПИСКУ ТОКЕНОВ
raw_pushkin_tokens = transform_to_list(pushkin_with_stopwords)


# СОЗДАНИЕ СЛОВАРЯ ГДЕ КЛЮЧ=СЛОВО, А ЗНАЧЕНИЕ=КОЛИЧЕСТВО ЕГО ПОВТОРЕНИЙ В ТЕКСТЕ
raw_pushkin_dict = Counter(raw_pushkin_tokens)


print("\n\nИЗ МАССИВА В КОТОРОМ СТОП-СЛОВА НЕ БЫЛИ УДАЛЕНЫ")
print(f"ПОЛУЧЕННЫЕ ТОКЕНЫ:\n{raw_pushkin_tokens[:5], raw_pushkin_tokens[-5:]}\n\n5 САМЫХ РАСПРОСТРАННЕННЫХ СЛОВ:\n{raw_pushkin_dict.most_common(5)}\n\n")



ИЗ МАССИВА В КОТОРОМ СТОП-СЛОВА НЕ БЫЛИ УДАЛЕНЫ
ПОЛУЧЕННЫЕ ТОКЕНЫ:
(['к', 'наталье', 'ъ', 'так', 'и'], ['без', 'чувств', 'в', 'ее', 'объятия'])

5 САМЫХ РАСПРОСТРАННЕННЫХ СЛОВ:
[('и', 10895), ('в', 6690), ('не', 4788), ('я', 3623), ('на', 3421)]


CPU times: user 552 ms, sys: 158 ms, total: 710 ms
Wall time: 4.99 s


# Сравнение двух словарей:
### 15 популярных слов с оставленными стоп-словами
### 15 популярных слов после удаления стоп-слов

In [ ]:
before = raw_pushkin_dict.most_common(15)
after = pushkin_dict.most_common(15)


print(f"Самые популярные слова\n{'-'*100}\n|{'до очистки:':^49}|{'после очистки:':^48}|\n{'-'*100}")

[print(f"|{v1[0]:^25}{v1[1]:^24}|{v2[0]:^24}{v2[1]:^24}|\n{'-'*100}") for v1, v2 in zip(before, after)]

Самые популярные слова
----------------------------------------------------------------------------------------------------
|                   до очистки:                   |                 после очистки:                 |
----------------------------------------------------------------------------------------------------
|            и                     10895          |          всё                     664           |
----------------------------------------------------------------------------------------------------
|            в                      6690          |         сказал                   449           |
----------------------------------------------------------------------------------------------------
|           не                      4788          |          это                     439           |
----------------------------------------------------------------------------------------------------
|            я                      3623          |          тебе   

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Стемминг и лемматизация

In [ ]:
# ДЛЯ ДАЛЬНЕЙШЕЙ ЛЕММАТИЗАЦИИ И СТЕММИНГА ИСПОЛЬЗУЕТСЯ ОЧИЩЕННЫЙ ОТ СТОП-СЛОВ СПИСОК
pushkin_tokens[:5], pushkin_tokens[-5:]

(['наталье', 'узнать', 'случилось', 'птица', 'купидон'],
 ['россии', 'вскрикнула', 'упала', 'чувств', 'объятия'])

## Стемминг

In [ ]:
%%time
# ИНИЦИАЛИЗАЦИЯ ИНСТРУМЕНТОВ ДЛЯ СТЕММИНГА
stemmer = SnowballStemmer('russian')

df = pd.DataFrame(pushkin_tokens, columns=["word"])

df["stemme"] = df["word"].parallel_apply(lambda x: stemmer.stem(x))

df.head()

CPU times: user 433 ms, sys: 112 ms, total: 545 ms
Wall time: 7.06 s


,word,stemme
0,наталье,натал
1,узнать,узна
2,случилось,случ
3,птица,птиц
4,купидон,купидон


# ---------------------------------------------------------------------------------------------------------------

In [ ]:
%%time

# СТЕММИНГ, ПРИВЕДЕНИЕ СЛОВА К БАЗОВОЙ ФОРМЕ, ОТБРАСЫВАНИЕ ОКОНЧАНИЙ
stemmes = [stemmer.stem(w) for w in tqdm(pushkin_tokens, desc='Stemming words')]

stemmes[:5], stemmes[-5:]

CPU times: user 4.4 s, sys: 18.8 ms, total: 4.42 s
Wall time: 4.42 s


(['натал', 'узна', 'случ', 'птиц', 'купидон'],
 ['росс', 'вскрикнул', 'упа', 'чувств', 'объят'])

In [ ]:
%%time
# ИНИЦИАЛИЗАЦИЯ ИНСТРУМЕНТОВ ДЛЯ СТЕММИНГА

df2 = pd.DataFrame(pushkin_tokens, columns=["word"])

df2["stemme"] = np.vectorize(lambda x: stemmer.stem(x))(df2["word"])

df2.head()

CPU times: user 4.63 s, sys: 31 ms, total: 4.66 s
Wall time: 4.68 s


,word,stemme
0,наталье,натал
1,узнать,узна
2,случилось,случ
3,птица,птиц
4,купидон,купидон


## Лемматизация

In [ ]:
%%time
# ИНИЦИАЛИЗАЦИЯ ИНСТРУМЕНТОВ ДЛЯ ЛЕММАТИЗАЦИИ
morph = pymorphy3.MorphAnalyzer()

# ЛЕММАТИЗАЦИЯ, ПРИВЕДЕНИЕ СЛОВА К БАЗОВОЙ ФОРМЕ
df["lemme"] = df["word"].parallel_apply(lambda x: morph.parse(x)[0].normal_form)

df.head()

CPU times: user 1.03 s, sys: 194 ms, total: 1.22 s
Wall time: 19.4 s


,word,stemme,lemme
0,наталье,натал,наталья
1,узнать,узна,узнать
2,случилось,случ,случиться
3,птица,птиц,птица
4,купидон,купидон,купидон


In [ ]:
%%time

# ЛЕММАТИЗАЦИЯ, ПРИВЕДЕНИЕ СЛОВА К БАЗОВОЙ ФОРМЕ
lemmes = [morph.parse(w)[0].normal_form for w in tqdm(pushkin_tokens, desc='Lemming words')]

lemmes[:5], lemmes[-5:]

CPU times: user 18.5 s, sys: 74.9 ms, total: 18.6 s
Wall time: 20.2 s


(['наталья', 'узнать', 'случиться', 'птица', 'купидон'],
 ['россия', 'вскрикнуть', 'упасть', 'чувство', 'объятие'])

In [ ]:
%%time

# ЛЕММАТИЗАЦИЯ, ПРИВЕДЕНИЕ СЛОВА К БАЗОВОЙ ФОРМЕ
df2["lemme"] = np.vectorize(lambda x: morph.parse(x)[0].normal_form)(df2["word"])

df2.head()

CPU times: user 19 s, sys: 63.8 ms, total: 19 s
Wall time: 19.2 s


,word,stemme,lemme
0,наталье,натал,наталья
1,узнать,узна,узнать
2,случилось,случ,случиться
3,птица,птиц,птица
4,купидон,купидон,купидон


In [ ]:
dframe = pd.DataFrame({
    'word': pushkin_tokens,
    'lemme': lemmes,
    'stemm': stemmes
})

dframe.head()

,word,lemme,stemm
0,наталье,наталья,натал
1,узнать,узнать,узна
2,случилось,случиться,случ
3,птица,птица,птиц
4,купидон,купидон,купидон


In [ ]:
# СОЗДАНИЕ СЛОВАРЯ СО СЛОВАМИ В БАЗОВОЙ ФОРМЕ
answer = Counter(lemmes)

answer.most_common(5)

[('свой', 1314), ('мой', 1078), ('сказать', 905), ('твой', 681), ('всё', 664)]

# Сравнение двух словарей:
### 15 популярных слов до лемматизации
### 15 популярных слов после лемматизации

In [ ]:
before = pushkin_dict.most_common(15)
after = answer.most_common(15)


print(f"Самые популярные слова\n{'-'*100}\n|{'до лемматизации:':^49}|{'после лемматизации:':^48}|\n{'-'*100}")
[print(f"|{v1[0]:^25}{v1[1]:^24}|{v2[0]:^24}{v2[1]:^24}|\n{'-'*100}") for v1, v2 in zip(before, after)]

Самые популярные слова
----------------------------------------------------------------------------------------------------
|                до лемматизации:                 |              после лемматизации:               |
----------------------------------------------------------------------------------------------------
|           всё                     664           |          свой                    1314          |
----------------------------------------------------------------------------------------------------
|         сказал                    449           |          мой                     1078          |
----------------------------------------------------------------------------------------------------
|           это                     439           |        сказать                   905           |
----------------------------------------------------------------------------------------------------
|          тебе                     380           |          твой   

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

# Расстояние Левенштейна

In [ ]:
def get_distance(word, bank):
    lword = len(word)
    sword = set(word)
    # abs(len(x) - len(word)) <= 2
    fltr = filter(lambda x: (lword - 1 <= len(x) <= lword + 1) and (len(sword - set(x)) <= 2), bank)
    return fltr


def get_similar(word, k, words):

    fltr = list(get_distance(word, words))
    print(f"Отфильтровал слова, оставил наиболее похожие, которые прошли фильтр:\n{fltr}\n\n")

    result = sorted(fltr, key=lambda x: edit_distance(word, x))[:k]
    print(f"Наиболее близкие по расстоянию Левенштейна слова к слову '{word}':\nКоличество - {k}:")
    return result


get_similar("Сабиров", 5, set(pushkin_tokens))

Отфильтровал слова, оставил наиболее похожие, которые прошли фильтр:
['избрав', 'виноград', 'собирала', 'бросали', 'прибавлю', 'стариков', 'собирая', 'воскраия', 'побраним', 'пировать', 'бросив', 'рифмова', 'бурцова', 'коровами', 'отвратив', 'обобрали', 'багровую', 'боязлива', 'изображу', 'ворами', 'рабство', 'рабством', 'прибавя', 'карлович', 'барков', 'багровый', 'радивою', 'товарищу', 'работали', 'вампиром', 'выбора', 'багровым', 'брантов', 'дубравой', 'отобрав', 'добавить', 'бородина', 'вихромах', 'коврами', 'ограбили', 'разбои', 'гробами', 'равниной', 'изорвала', 'боброва', 'оврагами', 'кровати', 'биографа', 'арбатове', 'квирога', 'поварихи', 'ограбил', 'ободрали', 'швабрин', 'говорила', 'обвилась', 'выборах', 'багровом', 'радивой', 'пивовар', 'раковин', 'оборван', 'бровями', 'обратила', 'рисовать', 'развозит', 'бородами', 'болтлива', 'раствори', 'прозвища', 'арбатову', 'требовал', 'пировали', 'вообрази', 'разобрав', 'христова', 'крапивой', 'швабрину', 'ратников', 'отправил', 'тиб

['сатиров', 'сабуров', 'сабурову', 'париков', 'избрав']

In [ ]:
get_similar("Сабиров", 5, answer)

Отфильтровал слова, оставил наиболее похожие, которые прошли фильтр:
['виноград', 'рисовать', 'обливать', 'работник', 'обратить', 'гробница', 'обрасти', 'багровый', 'пировать', 'товарищ', 'собрание', 'собирать', 'изорвать', 'возрасти', 'рабство', 'багров', 'отравить', 'обвивать', 'бритва', 'красиво', 'братство', 'прибавка', 'барство', 'дуброва', 'барков', 'обагрить', 'болгарин', 'тибров', 'правило', 'сабуров', 'обрание', 'амврозия', 'карлович', 'ариостов', 'кораблик', 'баронов', 'выбирать', 'ограбить', 'развитой', 'обшарить', 'рабочий', 'радивой', 'биограф', 'побратим', 'добавить', 'исправно', 'здаровие', 'оправить', 'бирало', 'пивовар', 'напротив', 'баловник', 'гаврилов', 'повариха', 'обдирать', 'арбатов', 'боливар', 'раковина', 'воскраий', 'швабрин', 'ворчание', 'отбирать']


Наиболее близкие по расстоянию Левенштейна слова к слову 'Сабиров':
Количество - 5:


['сабуров', 'багров', 'тибров', 'дуброва', 'барков']

In [ ]:
f = lambda: np.random.randint(0, len(pushkin_tokens))
five_pairs = [(pushkin_tokens[f()], pushkin_tokens[f()]) for i in range(5)]
five_pairs

[('виновата', 'ведет'),
 ('другие', 'пречистая'),
 ('нежный', 'время'),
 ('рукою', 'стихотворения'),
 ('сцену', 'такая')]

In [ ]:
[print(f"{'Расстояние редактирования между словами:':>30} {v1:^7} и {v2:^7} - {edit_distance(v1, v2):<20}") for v1, v2 in five_pairs]

Расстояние редактирования между словами: виновата и  ведет  - 6                   
Расстояние редактирования между словами: другие  и пречистая - 7                   
Расстояние редактирования между словами: нежный  и  время  - 6                   
Расстояние редактирования между словами:  рукою  и стихотворения - 12                  
Расстояние редактирования между словами:  сцену  и  такая  - 5                   


[None, None, None, None, None]

In [ ]:
edit_distance("Сабиров", "Сабуров")

1

#  
#  
#
#
#  
#  
#
#
#  
#  
#
#
#   
#  
#
#  
#
#
#  
#  
#
#
#   
#  
# ВТОРОЙ ФАЙЛ

In [ ]:
lt = pd.read_csv("https://raw.githubusercontent.com/ibbvs/jasa/main/TOBD-labs/data%20sources/LT1.txt",
                 encoding="windows-1251",
                 delimiter="\t")

lt.head()

,Л.Н. ТОЛСТОЙ
0,ВОЙНА И МИР
1,ТОМ ПЕРВЫЙ
2,ЧАСТЬ ПЕРВАЯ
3,I.
4,"- Еh bien, mon prince. Gкnes et Lucques ne sont plus que des apanages,"


In [ ]:
#ИНИЦИАЛИЗАЦИЯ СПИСКА СТОП-СЛОВ - русский и французкий

sw_rus = nltk.corpus.stopwords.words('russian')
sw_frn = nltk.corpus.stopwords.words('french')

sw_rus[:5], sw_frn[:5]

(['и', 'в', 'во', 'не', 'что'], ['au', 'aux', 'avec', 'ce', 'ces'])

In [ ]:
lt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46117 entries, 0 to 46116
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0       Л.Н. ТОЛСТОЙ  46117 non-null  object
dtypes: object(1)
memory usage: 360.4+ KB


In [ ]:
# Меняет все слова "вс„" на "всё
lt = lt.parallel_applymap(lambda x: re.sub("вс„", "всё", x.lower()))

In [ ]:
# УДАЛЯЕМ ЗНАКИ ПУНКТУАЦИИ, ФРАНЦУЗКИЙ ОСТАВЛЯЕМ
lt = remove_punkt(lt)

lt.head()

,Л.Н. ТОЛСТОЙ
0,война и мир
1,том первый
2,часть первая
3,i
4,еh bien mon prince gкnes et lucques ne sont plus que des apanages


In [ ]:
# СОХРАНЯЕМ ГРЯЗНЫЙ ФРЕЙМ, С СТОП-СЛОВАМИ

lt_with_stopwords = lt
lt_with_stopwords.head()

,Л.Н. ТОЛСТОЙ
46112,сноска открыть поприще
46113,сноска нитями богородицы
46114,сноска вы нездоровы
46115,сноска нет
46116,сноска общественный договор ред


In [ ]:
# УДАЛЯЕМ РУССКИЕ СТОП-СЛОВА
lt = lt.applymap(lambda x: remove_stopwords(x, sw_rus))

lt.head()

,Л.Н. ТОЛСТОЙ
0,война мир
1,первый
2,часть первая
3,
4,еh bien prince gкnes lucques plus apanages


In [ ]:
# УДАЛЯЕМ ФРАНЦУЗКИЕ СТОП-СЛОВА
lt = lt.applymap(lambda x: remove_stopwords(x, sw_frn))
lt.head()

,Л.Н. ТОЛСТОЙ
0,война мир
1,первый
2,часть первая
3,
4,еh bien prince gкnes lucques plus apanages


In [ ]:
# ПРИВОДИМ pd.lt К ОДНОМЕРНОМУ СПИСКУ, ПОЛУЧАЕМ СПИСОК ТОКЕНОВ
lt_tokens = transform_to_list(lt)
lt_tokens[:5], len(lt_tokens)

(['война', 'мир', 'первый', 'часть', 'первая'], 279829)

In [ ]:
# СОЗДАНИЕ СЛОВАРЯ ГДЕ КЛЮЧ=СЛОВО, А ЗНАЧЕНИЕ=КОЛИЧЕСТВО ЕГО ПОВТОРЕНИЙ В ТЕКСТЕ
lt_dict = Counter(lt_tokens)

lt_dict.most_common(5)

[('это', 2586),
 ('сказал', 2016),
 ('пьер', 1401),
 ('князь', 1354),
 ('всё', 979)]

In [ ]:
# ПРИВЕДЕНИЕ ГРЯЗНОГО ФРЕЙМА К ОДНОМЕРНОМУ СПИСКУ, СПИСКУ ТОКЕНОВ
raw_lt_tokens = transform_to_list(lt_with_stopwords)


# СОЗДАНИЕ СЛОВАРЯ ГДЕ КЛЮЧ=СЛОВО, А ЗНАЧЕНИЕ=КОЛИЧЕСТВО ЕГО ПОВТОРЕНИЙ В ТЕКСТЕ
raw_lt_dict = Counter(raw_lt_tokens)

raw_lt_tokens[:5], len(raw_lt_tokens), raw_lt_dict.most_common(5)

(['война', 'и', 'мир', 'том', 'первый'],
 466032,
 [('и', 21422), ('в', 11134), ('не', 8780), ('что', 8367), ('он', 7493)])

# Сравнение двух словарей:
### 15 популярных слов с оставленными стоп-словами
### 15 популярных слов после удаления стоп-слов

In [ ]:
before = raw_lt_dict.most_common(15)
after = lt_dict.most_common(15)


print(f"Самые популярные слова\n{'-'*100}\n|{'до очистки:':^49}|{'после очистки:':^48}|\n{'-'*100}")
[print(f"|{v1[0]:^25}{v1[1]:^24}|{v2[0]:^24}{v2[1]:^24}|\n{'-'*100}") for v1, v2 in zip(before, after)]

Самые популярные слова
----------------------------------------------------------------------------------------------------
|                   до очистки:                   |                 после очистки:                 |
----------------------------------------------------------------------------------------------------
|            и                     21422          |          это                     2586          |
----------------------------------------------------------------------------------------------------
|            в                     11134          |         сказал                   2016          |
----------------------------------------------------------------------------------------------------
|           не                      8780          |          пьер                    1401          |
----------------------------------------------------------------------------------------------------
|           что                     8367          |         князь   

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Стемминг и лемматизация

In [ ]:
# ДЛЯ ДАЛЬНЕЙШЕЙ ЛЕММАТИЗАЦИИ И СТЕММИНГА ИСПОЛЬЗУЕТСЯ СПИСОК ВСЕХ СЛОВ, КРОМЕ СТОП-СЛОВ
lt_tokens[:5], lt_tokens[-5:]

(['война', 'мир', 'первый', 'часть', 'первая'],
 ['сноска', 'сноска', 'общественный', 'договор', 'ред'])

## Стемминг

In [ ]:
%%time
# ИНИЦИАЛИЗАЦИЯ ИНСТРУМЕНТОВ ДЛЯ СТЕММИНГА
stemmer = SnowballStemmer('russian')

df = pd.DataFrame(lt_tokens, columns=["word"])

df["stemme"] = df["word"].parallel_apply(lambda x: stemmer.stem(x))

df.head()

CPU times: user 710 ms, sys: 199 ms, total: 909 ms
Wall time: 12.2 s


,word,stemme
0,война,войн
1,мир,мир
2,первый,перв
3,часть,част
4,первая,перв


In [ ]:
%%time

# СТЕММИНГ, ПРИВЕДЕНИЕ СЛОВА К БАЗОВОЙ ФОРМЕ, ОТБРАСЫВАНИЕ ОКОНЧАНИЙ
stemmes = [stemmer.stem(w) for w in tqdm(lt_tokens, desc='Stemming words')]

stemmes[:5]

Stemming words: 100%|██████████| 279829/279829 [00:09<00:00, 30915.90it/s]

CPU times: user 8.89 s, sys: 48.3 ms, total: 8.94 s
Wall time: 9.06 s


['войн', 'мир', 'перв', 'част', 'перв']

In [ ]:
%%time
df2 = pd.DataFrame(lt_tokens, columns=["word"])

df2["stemme"] = np.vectorize(lambda x: stemmer.stem(x))(df["word"])

df2.head()

CPU times: user 8.09 s, sys: 42.1 ms, total: 8.13 s
Wall time: 8.23 s


,word,stemme
0,война,войн
1,мир,мир
2,первый,перв
3,часть,част
4,первая,перв


## Лемматизация

In [ ]:
%%time
# ИНИЦИАЛИЗАЦИЯ ИНСТРУМЕНТОВ ДЛЯ ЛЕММАТИЗАЦИИ
morph = pymorphy3.MorphAnalyzer()

df["lemme"] = df["word"].parallel_apply(lambda x: morph.parse(x)[0].normal_form)

df.head()

CPU times: user 1.67 s, sys: 276 ms, total: 1.95 s
Wall time: 33.2 s


,word,stemme,lemme
0,война,войн,война
1,мир,мир,мир
2,первый,перв,первый
3,часть,част,часть
4,первая,перв,первый


In [ ]:
%%time

# ЛЕММАТИЗАЦИЯ, ПРИВЕДЕНИЕ СЛОВА К БАЗОВОЙ ФОРМЕ
lemmes = [morph.parse(w)[0].normal_form for w in tqdm(lt_tokens, desc='Lemming words')]

lemmes[:5]

Lemming words: 100%|██████████| 279829/279829 [00:45<00:00, 6171.82it/s]

CPU times: user 37.2 s, sys: 264 ms, total: 37.4 s
Wall time: 45.3 s


['война', 'мир', 'первый', 'часть', 'первый']

In [ ]:
%%time

# ЛЕММАТИЗАЦИЯ, ПРИВЕДЕНИЕ СЛОВА К БАЗОВОЙ ФОРМЕ
df2["lemme"] = np.vectorize(lambda x: morph.parse(x)[0].normal_form)(df2["word"])

df2.head()

CPU times: user 30.9 s, sys: 94.7 ms, total: 31 s
Wall time: 31.3 s


,word,stemme,lemme
0,война,войн,война
1,мир,мир,мир
2,первый,перв,первый
3,часть,част,часть
4,первая,перв,первый


In [ ]:
lt_frame = pd.DataFrame({
    'word': lt_tokens,
    'lemme': lemmes,
    'stemm': stemmes
})

lt_frame.head()

,word,lemme,stemm
0,война,война,войн
1,мир,мир,мир
2,первый,первый,перв
3,часть,часть,част
4,первая,первый,перв


In [ ]:
# СОЗДАНИЕ СЛОВАРЯ СО СЛОВАМИ В БАЗОВОЙ ФОРМЕ
lt_answer = Counter(lemmes)

lt_answer.most_common(5)

[('который', 3746),
 ('сказать', 3402),
 ('свой', 2804),
 ('это', 2774),
 ('пьер', 2183)]

# Сравнение двух словарей:
### 15 популярных слов до лемматизации
### 15 популярных слов после лемматизации

In [ ]:
before = lt_dict.most_common(15)
after = lt_answer.most_common(15)


print(f"Самые популярные слова\n{'-'*100}\n|{'до лемматизации:':^49}|{'после лемматизации:':^48}|\n{'-'*100}")
[print(f"|{v1[0]:^25}{v1[1]:^24}|{v2[0]:^24}{v2[1]:^24}|\n{'-'*100}") for v1, v2 in zip(before, after)]

Самые популярные слова
----------------------------------------------------------------------------------------------------
|                до лемматизации:                 |              после лемматизации:               |
----------------------------------------------------------------------------------------------------
|           это                     2586          |        который                   3746          |
----------------------------------------------------------------------------------------------------
|         сказал                    2016          |        сказать                   3402          |
----------------------------------------------------------------------------------------------------
|          пьер                     1401          |          свой                    2804          |
----------------------------------------------------------------------------------------------------
|          князь                    1354          |          это    

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

# Расстояние Левенштейна

In [ ]:
get_similar("Сабиров", 5, set(lt_tokens))

Отфильтровал слова, оставил наиболее похожие, которые прошли фильтр:
['избрав', 'выбирают', 'бросали', 'прибрано', 'стариков', 'поправив', 'собирая', 'бревнами', 'обратите', 'горчавки', 'рисковал', 'разложив', 'оправил', 'разбило', 'бросив', 'выронила', 'проживал', 'поправил', 'добывали', 'дворника', 'оттрубив', 'выбирал', 'красивое', 'выборам', 'призвано', 'правилом', 'обтирая', 'дворами', 'красивой', 'отбивали', 'багровый', 'подобрав', 'товарищу', 'работали', 'сворами', 'выбора', 'багровым', 'паровике', 'бородина', 'прибывая', 'разбивал', 'коврами', 'ограбили', 'гаврило', 'побрали', 'правило', 'кровати', 'выбирай', 'боярами', 'пробовал', 'говорила', 'оправить', 'выборах', 'аристово', 'обрывая', 'бровями', 'противна', 'затворив', 'обратила', 'бородами', 'баранов', 'побросав', 'требовал', 'обрывами', 'дровами', 'буровато', 'обливали', 'ратников', 'забросил', 'выиграно', 'кроватки', 'воротами', 'богатыри', 'отворила', 'багровое', 'пробками', 'братство', 'работами', 'оградив', 'выбирая',

['избрав', 'стариков', 'собирая', 'разбило', 'выбирал']

In [ ]:
get_similar("Сабиров", 5, answer)

Отфильтровал слова, оставил наиболее похожие, которые прошли фильтр:
['виноград', 'рисовать', 'обливать', 'работник', 'обратить', 'гробница', 'обрасти', 'багровый', 'пировать', 'товарищ', 'собрание', 'собирать', 'изорвать', 'возрасти', 'рабство', 'багров', 'отравить', 'обвивать', 'бритва', 'красиво', 'братство', 'прибавка', 'барство', 'дуброва', 'барков', 'обагрить', 'болгарин', 'тибров', 'правило', 'сабуров', 'обрание', 'амврозия', 'карлович', 'ариостов', 'кораблик', 'баронов', 'выбирать', 'ограбить', 'развитой', 'обшарить', 'рабочий', 'радивой', 'биограф', 'побратим', 'добавить', 'исправно', 'здаровие', 'оправить', 'бирало', 'пивовар', 'напротив', 'баловник', 'гаврилов', 'повариха', 'обдирать', 'арбатов', 'боливар', 'раковина', 'воскраий', 'швабрин', 'ворчание', 'отбирать']


Наиболее близкие по расстоянию Левенштейна слова к слову 'Сабиров':
Количество - 5:


['сабуров', 'багров', 'тибров', 'дуброва', 'барков']

In [ ]:
f = lambda: np.random.randint(0, len(lt_tokens))
five_pairs = [(lt_tokens[f()], lt_tokens[f()]) for i in range(5)]
five_pairs

[('давала', 'показалось'),
 ('цель', 'внимательным'),
 ('чехол', 'старому'),
 ('делали', 'приду'),
 ('года', 'недостойного')]

In [ ]:
[print(f"{'Расстояние редактирования между словами:':>30} {v1:^7} и {v2:^7} - {edit_distance(v1, v2):<20}") for v1, v2 in five_pairs]

Расстояние редактирования между словами: давала  и показалось - 7                   
Расстояние редактирования между словами:  цель   и внимательным - 9                   
Расстояние редактирования между словами:  чехол  и старому - 6                   
Расстояние редактирования между словами: делали  и  приду  - 6                   
Расстояние редактирования между словами:  года   и недостойного - 11                  


[None, None, None, None, None]

In [ ]:
edit_distance("птица", "феникс")

5